In [1]:
import os
import re
import pymatgen as mg
import pymatgen.analysis.diffraction as anadi
import pymatgen.analysis.diffraction.xrd as xrd
#import numpy as np
import glob
import matplotlib.pyplot as plt

In [2]:
import math
import time

In [3]:
import paddle_qcnn_5bits_5to4

In [4]:
import paddle
from paddle import fluid
from paddle_quantum.circuit import UAnsatz
from paddle_quantum.utils import partial_trace, dagger, state_fidelity, NKron,pauli_str_to_matrix
from paddle import complex
from paddle.complex import matmul, trace, kron,elementwise_add

In [5]:
import numpy as np
from sklearn import preprocessing

In [6]:
import xrd4QML
from xrd4QML import tomgStructure, get_xrdmat

In [7]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [8]:
torch.set_default_dtype(torch.float64)

In [9]:
person_path='/home/ipqi/.jupyter/20201020/'

train_path=person_path+'cpx222/qcnn/train/'

test_path=person_path+'cpx222/qcnn/test/'


In [10]:
def xrdpre_process(xrd_path):
    xrd1=xrd_path
    xrd2=tomgStructure(xrd1)
    xrd3=np.nan_to_num(get_xrdmat(xrd2))
    xrd4=preprocessing.normalize(np.array([xrd3]), norm="l2", axis=1)
    xrd5=np.dot(xrd4.T,xrd4)
    xrd6=xrd5.astype('complex128')
    return xrd6

In [11]:
trainfile=[]
for m1,n1,fname in os.walk(train_path):
    for ieach in n1:
        ieach=train_path+ieach
        trainfile.append(ieach)

In [12]:
testfile=[]
for m1,n1,fname in os.walk(test_path):
    for ieach in n1:
        ieach=test_path+ieach
        testfile.append(ieach)

In [13]:
#theta_load=np.asarray([1.05939, 2.85079, 5.95393, 3.59472, 2.57092, 5.31218, 2.38571, 2.06104, 6.08072, 3.40562, 7.28927, 1.09819, 2.89946, 0.0742845, 0.731151, 4.67235, -0.00228675, -0.131473, 3.13807, 3.36167, -0.042801])

In [14]:
LR=0.1
step=1
cstep=1
test_step=1
fid_train=[]
fid_test=[]
fid_train_s=[]
fid_test_s=[]
training_loss=[]
summary_test_correct=0
summary_train_correct=0

los_record=[]

with fluid.dygraph.guard():
    net1= paddle_qcnn_5bits_5to4.NET_2([21])
    
    opt = fluid.optimizer.Adam(learning_rate=LR, 
                          parameter_list=net1.parameters())
    
    for n in range(5):
        los_epoch=[]
        
        for i in trainfile:
            step=step+1
            input_xrd=xrdpre_process(i)
        
            out_net1, loss_net1 = net1(input_xrd)
            loss_net1.backward()
            opt.minimize(loss_net1)
            net1.clear_gradients()
        
            los_epoch.append(loss_net1) 
 
            fid = state_fidelity(input_xrd, out_net1.numpy())
        #training_loss.append(loss)
            if (step) % 50 ==0:
                print('iter:', step, 'loss:', '%.4f' % loss_net1, 'fid:', '%.4f' % fid)
        los_record.append(los_epoch)
    
    
    print('sumary-train')    
    
    for i in trainfile:
        
        
        input_xrd=xrdpre_process(i)
        
        out_net1, loss_net1 = net1(input_xrd)
        
        #train_loss, train_rho_out, input_data=net(trainxx)
        train_fid = state_fidelity(input_xrd, out_net1.numpy())
        fid_train_s.append(np.square(train_fid))
        fid_train.append(train_fid)
        
    print('sumary-test') 
    for i in testfile:
        #test_step=test_step+1
        
        input_xrd=xrdpre_process(i)
        
        out_net1, loss_net1 = net1(input_xrd)

        test_fid=state_fidelity(input_xrd, out_net1.numpy())
        fid_test_s.append(np.square(test_fid))
        fid_test.append(test_fid)


/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_

iter: 50 loss: 0.0207 fid: 0.9812


/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_

iter: 100 loss: 0.0184 fid: 0.9818


/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_

...

/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_

/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_

/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_

/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_

/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_

In [39]:
a=los_record[0]
print(a[0].numpy())

[0.65789816]


In [41]:
for i in los_record:
    print('---------------------start-------------------------')
    for ii in i:
        print(ii.numpy())

---------------------start-------------------------
[0.65789816]
[0.62994517]
[0.54008526]
[0.57259629]
[0.55554169]
[0.42357611]
[0.42066379]
[0.5254329]
[0.52852947]
[0.51966918]
[0.49372657]
[0.4495659]
[0.39373945]
[0.34287122]
[0.31126176]
[0.2873179]
[0.43562801]
[0.1820667]
[0.17031148]
[0.18063477]
[0.27127841]
[0.13294272]
[0.09763397]
[0.121952]
[0.08044342]
[0.05345592]
[0.13439302]
[0.0990278]
[0.05244536]
[0.04883628]
[0.03352489]
[0.03448827]
[0.04786545]
[0.05295164]
[0.04134807]
[0.03736607]
[0.04109651]
[0.05054371]
[0.0508227]
[0.04006749]
[0.03104162]
[0.02715568]
[0.03017759]
[0.03076036]
[0.02509674]
[0.0192435]
[0.01987442]
[0.02098338]
[0.02066144]
[0.01955088]
[0.01867645]
[0.03471403]
[0.03120306]
[0.01940123]
[0.02684732]
[0.02699464]
[0.02716174]
[0.02063646]
[0.02205105]
[0.02545365]
[0.02483493]
[0.02030467]
[0.01787785]
[0.02498518]
[0.02002475]
[0.01941753]
[0.02242643]
[0.02093457]
[0.02380169]
[0.02317082]
[0.02043221]
[0.01983432]
[0.02517739]
[0.01907

[0.01227516]
[0.01268732]
[0.01312546]
[0.01313061]
[0.01249621]
[0.01184007]
[0.01687262]
[0.01156086]
[0.01149208]
[0.01134059]
[0.01122261]
[0.01100267]
[0.01102223]
[0.01094406]
[0.01082929]
[0.02022127]
[0.01715714]
[0.01285023]
[0.01483495]
[0.01496632]
[0.01319593]
[0.01142931]
[0.01941249]
[0.01103496]
[0.02087979]
[0.01114464]
[0.01147714]
[0.01189654]
[0.01490617]
[0.01261598]
[0.01250577]
[0.01197318]
[0.01128669]
[0.01083856]
[0.01072022]
[0.01097182]
[0.01117448]
[0.01112286]
[0.01086998]
[0.01056143]
[0.02382211]
[0.01821899]
[0.01455803]
[0.01820025]
[0.01815734]
[0.0150084]
[0.01506454]
[0.01239715]
[0.01136621]
[0.01078488]
[0.01069108]
[0.01108407]
[0.01134468]
[0.0114246]
[0.01105902]
[0.01064778]
[0.02895041]
[0.01018992]
[0.0118114]
[0.01376124]
[0.01471551]
[0.01465773]
[0.01465757]
[0.01720095]
[0.01450376]
[0.01437465]
[0.01546379]
[0.0165552]
[0.011579]
[0.01126069]
[0.01824488]
[0.01094042]
[0.01759068]
[0.0116327]
[0.01564796]
[0.01292865]
[0.01321636]
[0.012

[0.01271961]
[0.0125273]
[0.01677318]
[0.01219425]
[0.01231862]
[0.0125102]
[0.01260219]
[0.01220094]
[0.01160177]
[0.01128115]
[0.01998302]
[0.01803355]
[0.01207568]
[0.0136797]
[0.01330274]
[0.01320958]
[0.01324063]
[0.0152904]
[0.0135664]
[0.01435696]
[0.01172636]
[0.01683221]
[0.01562948]
[0.01234273]
[0.01304147]
[0.01348844]
[0.01343692]
[0.01295119]
[0.01259629]
[0.01270044]
[0.01277596]
[0.0161518]
[0.01470873]
[0.01270008]
[0.01360084]
[0.01418949]
[0.01380921]
[0.01290701]
[0.01252251]
[0.01263376]
[0.01295409]
[0.01760947]
[0.01535755]
[0.01275011]
[0.014299]
[0.01490608]
[0.01403366]
[0.0127183]
[0.01237825]
[0.0128907]
[0.01327927]
[0.0192576]
[0.01215861]
[0.01198217]
[0.01260743]
[0.01308805]
[0.0128265]
[0.01204906]
[0.01135826]
[0.01832817]
[0.0177661]
[0.01171445]
[0.01260871]
[0.01288452]
[0.01237812]
[0.01160854]
[0.01764259]
[0.01113359]
[0.0111281]
[0.01114504]
[0.01106598]
[0.01093387]
[0.01077086]
[0.02002075]
[0.01093345]
[0.01162991]
[0.01184673]
[0.01134648]


In [29]:
print(net1.state_dict())

OrderedDict([('theta', name net_2_0.w_0, dtype: VarType.FP64 shape: [21] 	lod: {}
	dim: 21
	layout: NCHW
	dtype: double
	data: [1.10449 2.23501 3.04102 3.59472 2.57092 5.31218 3.34768 0.665225 5.56863 4.13427 6.38602 2.23355 2.89946 0.0742843 0.73115 4.61391 0.0718515 -0.59487 3.14137 3.16869 -0.0426613]
)])


In [15]:
print(net1.state_dict())

OrderedDict([('theta', name net_2_0.w_0, dtype: VarType.FP64 shape: [21] 	lod: {}
	dim: 21
	layout: NCHW
	dtype: double
	data: [1.50826 2.16304 3.54938 3.59472 2.57092 5.31218 3.19765 -0.364409 5.19505 4.66239 6.03654 2.37409 2.89946 0.0742845 0.731149 5.05812 2.37495 -0.540096 3.20501 3.07696 -1.39899]
)])


In [29]:
theta_load=np.asarray([1.50826, 2.16304, 3.54938, 3.59472, 2.57092, 5.31218, 3.19765, -0.364409, 5.19505, 4.66239, 6.03654, 2.37409, 2.89946, 0.0742845, 0.731149, 5.05812, 2.37495, -0.540096, 3.20501, 3.07696, -1.39899])

In [16]:
print(np.asarray(fid_test).mean())

0.9886707978588942


In [17]:
print(np.asarray(fid_train).mean())

0.9874857747699484


In [18]:
print(np.asarray(fid_test_s).mean())

0.9774732039633254


In [19]:
print(np.asarray(fid_train_s).mean())

0.975135390836359


In [19]:
print(np.asarray(fid_test).mean())

0.9853916300741077


In [23]:
print(np.asarray(fid_train).mean())

0.9856317957689914


In [21]:
print(np.asarray(fid_test_s).mean())

0.9709968076026947


In [22]:
print(np.asarray(fid_train_s).mean())

0.971470394704485


In [26]:
#

In [27]:
class Net_3(nn.Module):
    def __init__(self):
        super(Net_3, self).__init__()
        
        self.out=nn.Sequential(
            nn.Linear(4,32),
            nn.ReLU(),
            nn.Linear(32,1),            
        )
        
    def forward(self,x):
        x=x.view(x.size(0),-1)
        output=self.out(x)
        return output

In [37]:
#xrd4QML.get_energy(trainfile[0])

-122.69426

In [21]:
extend_num=1000
move_num=-122.69044

def linear_transform(energy):
    global extend_num, move_num
    energy_transform=(energy-move_num)*extend_num
    return energy_transform
def inverse_transform(energy_transform):
    global extend_num, move_num
    energy=energy_transform/extend_num+move_num
    return energy

In [30]:
loss_set=[]

with fluid.dygraph.guard():
    load_net1=paddle_qcnn_5bits_5to4.load_NET_2(theta_load)
    #load_net1=net1
    net3=Net_3()
    opt_net3=torch.optim.Adam(net3.parameters(),lr=0.01)
    
    for path_ in trainfile:
        try:
            total_energy=xrd4QML.get_energy(path_)
            tt_energy=linear_transform(total_energy)
        except:
            print(path_)
    
        input_xrd=xrdpre_process(path_)
    
        a_out,b_loss,c_encode=load_net1(input_xrd)
    
        input_net3=paddle_qcnn_5bits_5to4.measure_Z(c_encode)

        input_net3=np.asarray([[input_net3]])
        input_net3=np.asarray(input_net3,dtype=np.float64)
        input_net3=Variable(torch.from_numpy(input_net3),requires_grad=True)

        Gout=net3(input_net3)
        
        net3_loss=torch.abs(torch.mean(Gout-float(tt_energy)))
        
        opt_net3.zero_grad()
        net3_loss.backward()
        opt_net3.step()
    
        loss_set.append(net3_loss)


/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_

/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_

/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_

/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_

/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_

/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_

/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_

/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_

In [42]:
for i in loss_set:
    print(i.detach().numpy())

3.2813733953591955
0.7068777391957624
7.032268221132065
3.198111807671917
11.806595783412487
1.1474052428877561
0.7430076473168442
9.5537768030741
9.75303469448378
2.6877721863846187
9.372011446225638
6.197228061178078
2.315428986218798
2.596553964295887
4.000113508051142
2.1779561436745882
5.449335648107233
8.149886588250027
1.8739123264867419
11.937795940292325
5.995255264939452
11.716384145274464
9.252373793011074
2.848193603408869
2.6887118450021132
2.966930101462861
6.151528188941772
9.578639693558774
7.061728478348588
9.157953206755046
0.4174184370110199
4.068648610798188
10.231703327061634
3.6703788662914056
2.434612969739595
0.7506170832345864
3.9902946725155335
7.77822100363026
5.7288665978058715
1.0404927019325916
4.720696365171355
9.035373134848264
10.481915976537895
3.636957346849445
0.02665566479339443
3.73673165378886
3.7526915215098766
0.03032945143286092
3.661699802741836
3.8871956648178374
5.224488781913957
7.068289872306548
6.704121478193877
3.807374517346978
1.555898

In [ ]:
sss=[]
for i in loss_set:
    sss.append(i.detach().numpy())
sss=np.asarray(sss)

In [ ]:
sss.mean()

In [ ]:
help(loss_set[-1]

In [31]:

def get_energy_per_atom(energy):
    atoms_num=40
    energy_per_atom=energy/atoms_num
    return energy_per_atom

In [32]:
E_net3=[]
E_vasp=[]
abs_err=[]
mse_err=[]
with fluid.dygraph.guard():
    for path_ in testfile:
        #try:
        GGG=xrd4QML.get_energy(path_)
        
        GGG=get_energy_per_atom(GGG)
        E_vasp.append(GGG)
        input_xrd=xrdpre_process(path_)
        
        a_out,b_loss,c_encode=load_net1(input_xrd)
    
        input_net3=paddle_qcnn_5bits_5to4.measure_Z(c_encode)

        input_net3=np.asarray([[input_net3]])
        input_net3=np.asarray(input_net3,dtype=np.float64)
        input_net3=Variable(torch.from_numpy(input_net3),requires_grad=True)

        Gout=net3(input_net3)
        
        
        G_data=Gout.data.numpy().mean()
        G_data=inverse_transform(G_data)
        G_data=get_energy_per_atom(G_data)
        E_net3.append(G_data)
        #print(G_data)
        #print(GGG)
        abserr=abs(G_data-GGG)
        tmse=(G_data-GGG)**2
        mse_err.append(tmse)
        abs_err.append(abserr)
        #err0=abs(abserr/GGG)
        #err_t_0set.append(err0)
    #except:
        #print(path_)

/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_

/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_

/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_

/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_

/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_

/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_

/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_

/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_y/xrd_x
/home/ipqi/.jupyter/5to4-cpx222-b/xrd4QML.py:85: RuntimeWarning: invalid value encountered in true_divide
  xrd_z=xrd_

In [33]:
np.asarray(abs_err).mean()

0.00015383334684346828

In [34]:
np.asarray(mse_err).mean()

3.159755297298672e-08

In [55]:
np.asarray(mse_err).mean()

3.1395886310534926e-08

In [38]:
print(E_net3)

[-3.0673873194379078, -3.0673817156923255, -3.0673873929135445, -3.0673873929135445, -3.0673873929135445, -3.0673873929135445, -3.0673873562914062, -3.0673873929135445, -3.0673873562914062, -3.0673874654706337, -3.0673874654706337, -3.0673873194379078, -3.0673873194379078, -3.0673873194379078, -3.0673873562914062, -3.0673874654706337, -3.0673873194379078, -3.0673873562914062, -3.0673873562914062, -3.0673873562914062, -3.0673873194379078, -3.0673874654706337, -3.0673873562914062, -3.0673873929135445, -3.0673873194379078, -3.0673873194379078, -3.0673873562914062, -3.0673873562914062, -3.0673873194379078, -3.0673873929135445, -3.0673873194379078, -3.0673874654706337, -3.0673873929135445, -3.0673873562914062, -3.0673873562914062, -3.0673873194379078, -3.0673876078835822, -3.0673873929135445, -3.0673873194379078, -3.0673873562914062, -3.0673873929135445, -3.0673873929135445, -3.0673874654706337, -3.0673873562914062, -3.0673874654706337, -3.0673817156923255, -3.0673873929135445, -3.067387319

In [36]:
np.asarray(abs_err).mean()

1.0019290314047988

In [38]:
np.asarray(mse_err).mean()

4.738483954844378

In [37]:
1.0019290314047988/40

0.02504822578511997

In [39]:
4.738483954844378/1600

0.0029615524717777363

In [44]:
np.asarray(abs_err).mean()

0.00015302453264555728

In [45]:
np.asarray(mse_err).mean()

3.1295212558041735e-08